In [16]:
import joblib
import numpy as np
import pandas as pd

from haversine import haversine, Unit, inverse_haversine
from shapely.geometry import Point, LineString

# Load Pickled Model and Scaler

In [17]:
cam_meta = pd.read_csv("data/cam_meta_05302022.csv")
model = joblib.load('locating_angle_model.gz')
scaler = joblib.load('locating_scaler.gz')

In [18]:
def find_slope(angle, cl_angle, direction, xpix, xres):
    if direction == 'n':
        if xpix < (xres/2):
            return np.tan((np.pi/2)+angle+cl_angle)
        else:
            return np.tan((np.pi/2)-angle+cl_angle)
    if direction == 'e':
        if xpix < (xres/2):
            return np.tan(angle+cl_angle)
        else:
            return np.tan(-angle+cl_angle)
    if direction == 's':
        if xpix < (xres/2):
            return np.tan((3*np.pi/2)+angle+cl_angle)
        else:
            return np.tan((3*np.pi/2)-angle+cl_angle)
    if direction == 'w':
        if xpix < (xres/2):
            return np.tan(np.pi+angle+cl_angle)
        else:
            return np.tan(np.pi-angle+cl_angle)

In [19]:
class camera_view:
    def __init__(self, station, direction, tile_num, processed_xres=1856, processed_yres = 1040, precrop_yres = 1392, overlap_size=20, tile_size = [224,224]):
        self.station = station
        self.direction = direction
        self.tile_num = tile_num
        self.processed_xres = processed_xres
        self.processed_yres = processed_yres
        self.precrop_yres = precrop_yres
        self.overlap_size = overlap_size
        self.tile_size = tile_size
        self.full_xres = cam_meta['x_resolution'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]#get from metadata
        self.full_yres = cam_meta['y_resolution'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]#get from metadata
        self.elevation = cam_meta['elevation'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]
        self.cl_angle = cam_meta['center_angle'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]
        self.cam_lat = cam_meta['cam_lat'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]
        self.cam_long = cam_meta['cam_long'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]
        self.pix_centers = self.find_tile_centers()
        self.angle = self.find_angle()
        self.converted_angle = self.convert_angle()
                
    def find_tile_centers(self):
        num_tiles_x = int((self.processed_xres - self.overlap_size)/(self.tile_size[0] - self.overlap_size))
        num_tiles_y = int((self.processed_yres - self.overlap_size)/(self.tile_size[0] - self.overlap_size))
        full_y_crop = (self.precrop_yres / self.processed_yres)
        xres_ratio = self.full_xres/self.processed_xres
        yres_ratio = (self.full_yres*(self.processed_yres/self.precrop_yres))/self.processed_yres
        stepsize_x = (self.tile_size[0]*xres_ratio) - (self.overlap_size * xres_ratio)
        stepsize_y = (self.tile_size[1]*yres_ratio) - (self.overlap_size * yres_ratio)
        x_pix = [round(((self.tile_size[0]*xres_ratio)/2) + (i*stepsize_x)) for i in range(num_tiles_x)]
        y_pix = [round(((self.tile_size[1]*yres_ratio)/2) + (i*stepsize_y) + (((self.precrop_yres-self.processed_yres) / self.precrop_yres)*self.full_yres)) for i in range(num_tiles_y)]
        
        return [[x_pix[j],y_pix[i]] for i in range(len(y_pix)) for j in range(len(x_pix))]
    
    def find_angle(self):
        x_ratio = self.pix_centers[self.tile_num][0] / self.full_xres
        y_ratio = self.pix_centers[self.tile_num][1] / self.full_yres
        ang1 = model.predict(scaler.transform([[x_ratio, y_ratio, self.elevation]]))
        return ang1
    
    def convert_angle(self):
        dir_dict = {'n': np.pi/2, 'w': np.pi, 's': 3*np.pi/2, 'e': 0}
        if (self.pix_centers[self.tile_num][0]) <= (self.full_xres/2):
            return  self.cl_angle + dir_dict[self.direction] + self.angle
        else:
            return  self.cl_angle + dir_dict[self.direction] - self.angle
        
    def single_camera(self):
        cam_coord = (self.cam_lat, self.cam_long)
        cam_end = inverse_haversine(cam_coord, 30, self.angle)
        left_start = inverse_haversine(cam_coord, 3, self.angle+(np.pi/2))
        left_end = inverse_haversine(left_start, 30, self.angle)
        right_start = inverse_haversine(cam_coord, 3, self.angle-(np.pi/2))
        right_end = inverse_haversine(right_start, 30, self.angle)

        line_left = LineString([cam_coord, cam_end])
        line_center = LineString([left_start, left_end])
        line_right = LineString([right_start, right_end])

        return gpd.GeoDataFrame([line_left, line_center, line_right],  columns=['LineString_obj'],  geometry='LineString_obj')

# Create Camera View Objects

In [20]:
cv1 = camera_view(station='bh', direction='e', tile_num=11)
cv2 = camera_view(station='hp', direction='w', tile_num=8)

cv_list = [cv1, cv2]

# Check for Intersections

In [67]:
df_intersections = pd.read_csv("data/intersections.csv")

In [68]:
df_filt = df_intersections[df_intersections[['station', 'dir']].apply(tuple, axis=1).isin([(cv.station, cv.direction) for cv in cv_list])]

In [69]:
df_filt

,hpweren_camera_description,station,dir,cam_lat,cam_long,fov_right,fov_left,intersections,num_intersection
17,Boucher Hill,bh,e,33.334628,-116.919328,"(-116.6416284, 33.05692774)","(-116.6416284, 33.612327740000005)","[('bm', 'n'), ('bm', 'e'), ('bl', 'n'), ('bl',...",30
35,High Point,hp,w,33.363018,-116.836220,"(-117.11392, 33.640718)","(-117.11392, 33.085317999999994)","[('bm', 'n'), ('bm', 'w'), ('bl', 'n'), ('buff...",25


In [76]:
pairs = []
for i, base_cv in enumerate(cv_list):
    for cv in cv_list:
        compare_cam = str((cv.station, cv.direction))
        if compare_cam in df_filt['intersections'].iloc[i]:
            pairs.append({base_cv, cv})
            
tri_pairs = [i for n, i in enumerate(pairs) if i not in pairs[:n]]

# Perform Triangulation

In [77]:
tri_pairs

[{<__main__.camera_view at 0x2278f3be820>,
  <__main__.camera_view at 0x2278f3bed60>}]

In [80]:
for pair in tri_pairs:
    lpair = list(pair)
    lc1= lpair[0]
    lc2 = lpair[1]
    
    m1 = find_slope(lc1.angle, lc1.cl_angle, lc1.direction, lc1.pix_centers[lc1.tile_num][0], lc1.full_xres)
    b1 = lc1.cam_lat - (m1 * lc1.cam_long)
    m2 = find_slope(lc2.angle, lc2.cl_angle, lc2.direction, lc2.pix_centers[lc2.tile_num][0], lc2.full_xres)
    b2 = lc2.cam_lat - (m2 * lc2.cam_long)

    x_coord = (b2 - b1)/(m1-m2)
    y_coord = (m1 * x_coord) + b1

    print("Fire Estimate between {} and {} Stations: {}".format(lc1.station, lc2.station, (x_coord[0], y_coord[0])))

Fire Estimate between hp and bh Stations: (-116.82708655818011, 33.35306545842778)
